# Schema template

## Import

In [1]:
import json
import pandas as pd

## Files

In [17]:
infile = "Input/ISCIII_reception.xlsx"
pd.read_excel(infile, sheet_name="Datos", header=0)

,ID CNM,Código CCAA,Comunidad autónoma,Hospital,Médico,Código SiViEs,Ref Hospital,Fecha de toma,Fecha recepción,Semana ISO Fecha toma,...,S982,T1027,Q1071,E1092,H1101,E1111,D1118,V1176,G1219,SARS-CoV-2 Wuhan-Hu-1 NC_045512 (21563..25384)
0,202074288.0,MC,MURCIA,HOSPITAL UNIVERSITARIO VIRGEN DE LA ARRIXACA,"MORENO DOCON, ANTONIO",No consta,16065902,2020-12-20,2020-12-22,51,...,A,.,.,.,.,.,H,.,.,NaN
1,202074526.0,MC,MURCIA,HOSPITAL UNIVERSITARIO VIRGEN DE LA ARRIXACA,"MORENO DOCON, ANTONIO",No consta,11522092,2020-12-22,2020-12-23,52,...,.,.,.,.,.,.,.,.,.,NaN
2,202074508.0,CM,CASTILLA LA MANCHA,CONSEJERIA DE SANIDAD Y ASUNTOS SOCIALES,"GUTIERREZ AVILA, GONZALO",No consta,558955,2020-12-21,2020-12-23,52,...,A,.,.,.,.,.,H,.,.,NaN
3,202074512.0,CM,CASTILLA LA MANCHA,CONSEJERIA DE SANIDAD Y ASUNTOS SOCIALES,"GUTIERREZ AVILA, GONZALO",No consta,558954,2020-12-21,2020-12-23,52,...,A,.,.,.,.,.,H,.,.,NaN
4,202074444.0,GA,GALICIA,COMPLEJO HOSPITALARIO UNIVERSITARIO DE SANTIAGO,"PEÑA RODRIGUEZ, FERNANDA",No consta,74007341,2020-12-15,2020-12-28,51,...,?,?,.,.,.,.,H,.,.,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9304,202217774.0,MC,MURCIA,HOSPITAL DE LA VEGA LORENZO GUIRAO,"ALVAREZ MELGARES DE AGUILAR, JOAQUIN",No consta,91037147,2022-04-19,2022-05-03,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9305,202217773.0,MC,MURCIA,HOSPITAL DE LA VEGA LORENZO GUIRAO,"ALVAREZ MELGARES DE AGUILAR, JOAQUIN",No consta,90115943,2022-04-20,2022-05-03,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9306,202217759.0,MC,MURCIA,HOSPITAL DE LA VEGA LORENZO GUIRAO,"ALVAREZ MELGARES DE AGUILAR, JOAQUIN",No consta,92111719,2022-03-17,2022-05-03,11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9307,202217755.0,MC,MURCIA,HOSPITAL DE LA VEGA LORENZO GUIRAO,"ALVAREZ MELGARES DE AGUILAR, JOAQUIN",No consta,91034292,2022-02-28,2022-05-03,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## General schemas

### Funcionamiento básico

* Recepción del archivo
* Detección del laboratorio del que se trata
    * Loop por las claves del diccionario de laboratorios, si se detecta uno, se toma por ese
    * En caso de no encontrar ninguna, usar questionnaire para que indique cuál es? **O RAISE ERROR**
    * En caso de encontrar varias, usar questionnaire para seleccionar cuál es?
* Selección del _json_file_path_ usando el diccionario
* Dos opciones:
    * Modificar la dataframe para hacerla la dataframe final
    * Crear la dataframe final e ir rellenándola con la dataframe inicial
* Profit
    


In [20]:
lab_dict = {
            "lab_name" : "json_file_path",
            "lab2_name" : "json_file_path",
            "lab3_name" : "json_file_path",
            "lab4_name" : None,
            }

term_dict = {
    "equivalences" : {"termino_final" : "termino_inicial", 
                      "termino_final_2" : "termino_inicial_2"},
    "constants" : {"termino_final" : "valor constante"},
    "empty" : ["terminos_finales_no_incluidos"],
    "outer" : {"termino_final" : {"localización" : "término_inicial"}}
            }


In [ ]:
final_format = [
    "Public Health sample id (SIVIES)",
    "Sample ID given by originating laboratory",
    "Sample ID given by the submitting laboratory",
    "Sample ID given in the microbiology lab",
    "Sample ID given if multiple rna-extraction or passages",
    "Sample ID given for sequencing",
    "ENA Sample ID",
    "GISAID Virus Name",
    "GISAID id",
    "Originating Laboratory",
    "Submitting Institution",
    "Sample Collection Date",
    "Sample Received Date",
    "Purpose of sampling",
    "Biological Sample Storage Condition ",
    "Specimen source",
    "Environmental Material",
    "Environmental System",
    "Collection Device",
    "Host",
    "Host Age",
    "Host Gender",
    "Sequencing Date",
    "Rna Extraction Protocol",
    "Commercial All-in-one library kit",
    "Library Preparation Kit",
    "Enrichment Protocol",
    "If Enrichment Protocol. If Other,Specify",
    "Enrichment panel/assay",
    "If Enrichment panel/assay. If Other, Specify",
    "Enrichment panel/assay version",
    "Number Of Samples In Run",
    "Runid",
    "Sequencing Instrument Model",
    "Flowcell Kit",
    "Source material",
    "Capture method",
    "Sequencing technique",
    "Library Layout",
    "Gene Name 1",
    "Diagnostic Pcr Ct Value 1",
    "Gene Name 2",
    "Diagnostic Pcr Ct Value-2",
    "Analysis Authors",
    "Author Submitter",
    "Authors",
    "Sequence file R1 fastq",
    "Sequence file R2 fastq"
]

## ISCIII schemas

In [14]:
term_dict_ISCIII = {
    "equivalences" : {"Public Health sample id" : "Código SiViEs",
                     "Sample ID given by originating laboratory" : "Ref Hospital",
                     "Sample ID given by the submitting laboratory" : "ID CNM",
                     "Sample ID given in the microbiology lab" : "ID VI-VRP",
                     "Sample ID given if multiple rna-extraction or passages" : "ID VI-VRP",
                     "Sample ID given for sequencing" : "ID VI-VRP",
                     "GISAID id" : "ID GISAID",
                     "Originating Laboratory" : "Hospital",
                     "Sample Collection Date" : "Fecha de toma ",
                     "Sample Received Date" : "Fecha recepción",
                     "Host Age" : "Grupo edad",
                     "Diagnostic Pcr Ct Value 1" : "PCR genE",
                     "Sequencing Date" : "Fecha de secuenciación",
                     "Rna Extraction Protocol" : ""                     
                    },
    "constants" : {},
    "empty" : [],
    "outer" : { }
            }

In [18]:
term_dict_ISCIII

{'equivalence': {'Public Health sample id': 'Código SiViEs',
  'Sample ID given by originating laboratory': 'Ref Hospital',
  'Sample ID given by the submitting laboratory': 'ID CNM',
  'Sample ID given in the microbiology lab': 'ID VI-VRP',
  'Sample ID given if multiple rna-extraction or passages': 'ID VI-VRP',
  'Sample ID given for sequencing': 'ID VI-VRP',
  'GISAID id': 'ID GISAID',
  'Originating Laboratory': 'Hospital',
  'Sample Collection Date': 'Fecha de toma ',
  'Sample Received Date': 'Fecha recepción',
  'Host Age': 'Grupo edad',
  'Diagnostic Pcr Ct Value 1': 'PCR genE',
  'Sequencing Date': 'Fecha de secuenciación',
  'Rna Extraction Protocol': ''},
 'constants': {},
 'empty': [],
 'outer': {}}